## 1. Усердные ученики

In [302]:
query = """
SELECT
    COUNT(*) as diligent_students
FROM
    (SELECT 
         st_id
    FROM default.peas
    WHERE
        toStartOfMonth(timest) == '2021-10-01' -- фильтруем по нужной дате
    GROUP BY
         st_id
    HAVING
         SUM(correct) >= 20)
"""

## 2. Оптимизация воронки

In [303]:
query = """
SELECT
    test_grp,
    round(sum(revenue) / uniqExact(st_id), 2) as ARPU,
    round(sumIf(revenue, active_stud=1) / uniqExactIf(st_id, active_stud=1), 2) as ARPAU,
    round(uniqExactIf(st_id, revenue > 0) / uniqExact(st_id) * 100, 2) as CR_to_buy,
    round(uniqExactIf(st_id, active_stud=1 and revenue>0) / uniqExactIf(st_id, active_stud=1) * 100, 2) as CR_active,
    round(uniqExactIf(st_id, math_active_stud=1 and math_revenue>0)/uniqExactIf(st_id, math_active_stud=1) * 100, 2) as CR_math
FROM
    (
    SELECT 
        l.test_grp as test_grp,
        l.st_id as st_id,
        r.active_stud as active_stud,
        r.math_active_stud as math_active_stud,
        r.revenue as revenue,
        r.math_revenue as math_revenue
    FROM
        (
        SELECT
            test_grp,
            st_id
        FROM default.studs
        ) as l
    LEFT JOIN
        (
        SELECT
            l.st_id as st_id,
            l.active_stud as active_stud,
            l.math_active_stud as math_active_stud,
            r.revenue as revenue,
            r.math_revenue as math_revenue
        FROM
            (
            SELECT
                st_id,
                if (sum(correct) > 10, 1, 0) as active_stud,
                if (sumIf(correct, subject='Math') >= 2, 1, 0) as math_active_stud
            FROM default.peas
            GROUP BY
                st_id
            ) as l
        LEFT JOIN
            (
            select
                st_id,
                sum(money) as revenue,
                sumIf(money, subject='Math') as math_revenue
            from default.final_project_check
            group by st_id
            ) as r
        ON l.st_id = r.st_id
        ) as r
    ON l.st_id = r.st_id
    )
GROUP BY
    test_grp
"""